# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import warnings
warnings.filterwarnings('ignore')

# import libraries
import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pickle

import nltk
#nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords', 'ignore'])
nltk.download(['punkt', 'wordnet', 'stopwords', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator, TransformerMixin

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database with read_sql_table
engine = create_engine('sqlite:///DRP_Messages.db')
df = pd.read_sql_table('DRP_Messages', con = engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# take a look at the data
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,...,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,...,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,...,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
# drop child_alone since it is all zeros
df = df.drop('child_alone', axis = 1)

# replace the 2's in related with 1's - assuming these are errors
#df['related'] = df['related'].map(lambda x: 1 if x==2 else x)
df['related'] = df['related'].replace(2, 1)

In [5]:
#Define feature and target variables X and Y
X = df['message']
#y = df.iloc[:,4:]
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    """
    Clean and tokenize the text data
    
    Input: text - text data that needs to be cleaned and tokenized
    
    Output: clean_tokens - list of tokens extracted from the text data
    """
    
    #regular expression to detect a url
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
        
    # normalize text    
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stopwords.words("english")]
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:     
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

In [7]:
# test out function
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['hurricane'] 

Looking for someone but no name
['looking', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
# Evaluation metrics for training data
y_pred_train = pipeline.predict(X_train)

# Print the report on f1 score, precision and recall for each output category
print(classification_report(y_train.values, y_pred_train, target_names = y.columns.values))

                        precision    recall  f1-score   support

               related       0.99      1.00      0.99     15049
               request       1.00      0.92      0.96      3371
                 offer       1.00      0.67      0.80        87
           aid_related       1.00      0.97      0.98      8153
          medical_help       1.00      0.86      0.92      1573
      medical_products       1.00      0.85      0.92       986
     search_and_rescue       1.00      0.77      0.87       536
              security       1.00      0.76      0.86       352
              military       1.00      0.86      0.92       654
                 water       1.00      0.92      0.96      1247
                  food       1.00      0.96      0.98      2212
               shelter       1.00      0.91      0.95      1741
              clothing       1.00      0.85      0.92       316
                 money       1.00      0.78      0.88       459
        missing_people       1.00      

In [11]:
# Evaluation metrics for test data
y_pred_test = pipeline.predict(X_test)

# Print the report on f1 score, precision and recall for each output category
print(classification_report(y_test.values, y_pred_test, target_names = y.columns.values))

                        precision    recall  f1-score   support

               related       0.85      0.92      0.88      5045
               request       0.80      0.43      0.56      1103
                 offer       0.00      0.00      0.00        31
           aid_related       0.75      0.60      0.67      2707
          medical_help       0.55      0.08      0.15       511
      medical_products       0.72      0.07      0.13       327
     search_and_rescue       0.50      0.04      0.07       188
              security       0.67      0.02      0.03       119
              military       0.45      0.11      0.17       206
                 water       0.87      0.32      0.47       425
                  food       0.79      0.60      0.69       711
               shelter       0.77      0.28      0.41       573
              clothing       0.64      0.10      0.17        89
                 money       0.50      0.03      0.06       145
        missing_people       0.00      

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
# See parameters of pipeline
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fcec74949d8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [13]:
# Running grid search can take a while, especially if you are searching over a lot of parameters!
# Therefore I have limited the number of parameters in my grid search

#specify parameters for grid search
parameters = {'clf__estimator__min_samples_split': [3, 4],
             'clf__estimator__n_estimators': [20, 40]}

# create a grid search object
cv = GridSearchCV(pipeline, param_grid = parameters)

cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__min_samples_split': [3, 4], 'clf__estimator__n_estimators': [20, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
# Retesting the model using the grid search results

y_pred_test = cv.predict(X_test)

# Print the report on f1 score, precision and recall for each output category
print(classification_report(y_test.values, y_pred_test, target_names = y.columns.values))

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      5045
               request       0.81      0.48      0.60      1103
                 offer       0.00      0.00      0.00        31
           aid_related       0.74      0.71      0.72      2707
          medical_help       0.65      0.09      0.15       511
      medical_products       0.87      0.12      0.21       327
     search_and_rescue       1.00      0.03      0.05       188
              security       1.00      0.02      0.03       119
              military       0.68      0.11      0.19       206
                 water       0.91      0.39      0.54       425
                  food       0.83      0.65      0.73       711
               shelter       0.81      0.39      0.53       573
              clothing       0.86      0.07      0.12        89
                 money       0.80      0.03      0.05       145
        missing_people       0.00      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
# Change the classifier to AdaBoostClassifier
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])


pipeline2.fit(X_train, y_train)
y_updated_pred_test = pipeline2.predict(X_test)

In [16]:
# Print the report on f1 score, precision and recall for each output category
print(classification_report(y_test.values, y_updated_pred_test, target_names = y.columns.values))

                        precision    recall  f1-score   support

               related       0.80      0.97      0.88      5045
               request       0.74      0.50      0.60      1103
                 offer       0.00      0.00      0.00        31
           aid_related       0.76      0.61      0.68      2707
          medical_help       0.57      0.28      0.38       511
      medical_products       0.67      0.30      0.42       327
     search_and_rescue       0.60      0.18      0.28       188
              security       0.22      0.04      0.07       119
              military       0.57      0.31      0.40       206
                 water       0.74      0.65      0.69       425
                  food       0.80      0.64      0.71       711
               shelter       0.75      0.54      0.62       573
              clothing       0.67      0.35      0.46        89
                 money       0.54      0.26      0.35       145
        missing_people       0.47      

### 9. Export your model as a pickle file

In [17]:
# export the model as a pickle file
# the model from the grid search seem to do the best
pickle.dump(cv, open('my_final_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.